In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dataset import Dataset
import numpy as np

In [3]:
from common import *

In [4]:
import torch.nn as nn

In [ ]:
data = Dataset(name='Readmission', dirname='preprocess/Readmission/')
labellist = ['y']
data.generate_labels(labellist, len(labellist), 'binary')

In [ ]:
data.generate_encoded_field('gender_y', 'onehot')
data.generate_encoded_field('age_y', 'onehot')
data.generate_encoded_field('ethnicity_y', 'onehot')

In [ ]:
features = [x for x in data.dataframe.columns if x.startswith('feature')]
for f in features :
    data.generate_encoded_field(f, 'trivial')
    
data.set_structured_params(regexs=[r'^feature', 'gender_y', 'age_y', 'ethnicity_y'])

In [ ]:
from trainer import Trainer, Evaluator

In [ ]:
from models.Vanilla import ClassificationTrainer as BasicCT
from models.hierarchical import ClassificationTrainer as HierCT

In [ ]:
from Experiments.experiments import experiments, hierarchical_experiments, structured_experiments

Basic Experiments
=================

In [ ]:
train_data = data.get_data('train', structured=True)
dev_data = data.get_data('dev', structured=True)

In [ ]:
train_data = data.filter_data_length(train_data, truncate=90)
dev_data = data.filter_data_length(dev_data, truncate=90)

In [ ]:
for e in experiments :
    config = e(data, structured=False)
    print(config)
    trainer = Trainer(BasicCT, config)
    trainer.train(train_data, dev_data)

    evaluator = Evaluator(BasicCT, trainer.model.dirname)
    _ = evaluator.evaluate(dev_data, save_results=True)
    print('='*30)
    
for e in hierarchical_experiments :
    config = e(data, structured=False)
    print(config)
    trainer = Trainer(HierCT, config)
    trainer.train(train_data, dev_data)

    evaluator = Evaluator(HierCT, trainer.model.dirname)
    _ = evaluator.evaluate(dev_data, save_results=True)
    print('='*30)

In [ ]:
for e in experiments :
    config = e(data, structured=True)
    print(config)
    trainer = Trainer(BasicCT, config)
    trainer.train(train_data, dev_data)

    evaluator = Evaluator(BasicCT, trainer.model.dirname)
    _ = evaluator.evaluate(dev_data, save_results=True)
    print('='*30)
    
for e in hierarchical_experiments :
    config = e(data, structured=True)
    print(config)
    trainer = Trainer(HierCT, config)
    trainer.train(train_data, dev_data)

    evaluator = Evaluator(HierCT, trainer.model.dirname)
    _ = evaluator.evaluate(dev_data, save_results=True)
    print('='*30)

In [ ]:
train_data = data.get_data('train', structured=True, encodings=['gender_y', 'ethnicity_y', 'age_y'])
dev_data = data.get_data('dev', structured=True, encodings=['gender_y', 'ethnicity_y', 'age_y'])

train_data = data.filter_data_length(train_data, truncate=90)
dev_data = data.filter_data_length(dev_data, truncate=90)

In [ ]:
for e in structured_experiments :
    config = e(data, structured=True, encodings=['gender_y', 'ethnicity_y', 'age_y'])
    print(config)
    trainer = Trainer(BasicCT, config)
    trainer.train(train_data, dev_data)

    evaluator = Evaluator(BasicCT, trainer.model.dirname)
    _ = evaluator.evaluate(dev_data, save_results=True)
    print('='*30)

In [ ]:
train_data = data.get_data('train', structured=True, encodings=['gender_y', 'ethnicity_y', 'age_y'])
dev_data = data.get_data('dev', structured=True, encodings=['gender_y', 'ethnicity_y', 'age_y'])

train_data = data.filter_data_length(train_data, truncate=90)
dev_data = data.filter_data_length(dev_data, truncate=90)

for e in structured_experiments :
    config = e(data, structured=False, encodings=['gender_y', 'ethnicity_y', 'age_y'])
    print(config)
    trainer = Trainer(BasicCT, config)
    trainer.train(train_data, dev_data)

    evaluator = Evaluator(BasicCT, trainer.model.dirname)
    _ = evaluator.evaluate(dev_data, save_results=True)
    print('='*30)

In [ ]:
train_data = data.get_data('train', structured=True, encodings=data.structured_columns)
dev_data = data.get_data('dev', structured=True, encodings=data.structured_columns)
train_data = data.filter_data_length(train_data, truncate=90)
dev_data = data.filter_data_length(dev_data, truncate=90)

for e in structured_experiments :
    config = e(data, structured=True, encodings=data.structured_columns)
    print(config)
    
    trainer = Trainer(BasicCT, config)
    trainer.train(train_data, dev_data)

    evaluator = Evaluator(BasicCT, trainer.model.dirname)
    _ = evaluator.evaluate(dev_data, save_results=True)
    print('='*30)

In [ ]:
train_data = data.get_data('train', structured=True, encodings=data.structured_columns)
dev_data = data.get_data('dev', structured=True, encodings=data.structured_columns)
train_data = data.filter_data_length(train_data, truncate=90)
dev_data = data.filter_data_length(dev_data, truncate=90)

for e in structured_experiments :
    config = e(data, structured=False, encodings=data.structured_columns)
    print(config)
    
    trainer = Trainer(BasicCT, config)
    trainer.train(train_data, dev_data)

    evaluator = Evaluator(BasicCT, trainer.model.dirname)
    _ = evaluator.evaluate(dev_data, save_results=True)
    print('='*30)

Baselines
==========

In [ ]:
train_data = data.get_data('train', structured=True)
dev_data = data.get_data('dev', structured=True)

In [ ]:
from PatientVec.models.baselines.LR import LR, LDA

In [ ]:
lr = LR({'vocab' : data.vocab, 'stop_words' : True, 'exp_name' : data.name})
lr.train(train_data)
lr.evaluate(dev_data, save_results=True)
# lr.get_features(n=20)

In [ ]:
lda = LDA({'vocab' : data.vocab, 'stop_words' : True, 'exp_name' : data.name})
lda.train(train_data)
lda.evaluate(dev_data, save_results=True)
print(lda.get_topics(n=10))
topics = lda.get_topics(n=10)
print([topics[i] for i in np.argsort(lda.lda_classifier.coef_[0])])

In [ ]:
[topics[i] for i in np.argsort(lda.lda_classifier.coef_[0])]

In [ ]:
preds = outputs['predictions'][:, 0]

In [ ]:
from common import collapse_and_print_word_attn, print_sent_attn

In [ ]:
n = 50
collapse_and_print_word_attn(data.vocab, dev_data.X[n], outputs['word_attentions'][n])
print_sent_attn(data.vocab, dev_data.X[n], outputs['sentence_attentions'][n])

In [ ]:
outputs['sentence_attentions'][0]

In [ ]:
from scipy.stats import spearmanr, kendalltau

In [ ]:
corrs = [kendalltau(range(len(outputs['sentence_attentions'][i])), outputs['sentence_attentions'][i]) 
         for i in range(len(outputs['sentence_attentions']))]

In [ ]:
rho, leng = zip(*[(x[0], y) for x, y in zip(corrs, [len(z) for z in outputs['sentence_attentions']]) if x[0] == x[0]])

In [ ]:
plt.hist(rho, bins=30)

In [ ]:
pval, leng1 = zip(*[(x[1], y) for x, y in zip(corrs, [len(z) for z in outputs['sentence_attentions']]) if x[1] == x[1]])

Saving Models
==============

In [ ]:
import os
for e in experiments :
    config = e(data, structured=False)
    filename = config['exp_config']['exp_name']
    filename = os.path.join('outputs/classification/', filename)
    push_latest_model(filename, config['exp_config']['exp_name'])
    
    config = e(data, structured=True)
    filename = config['exp_config']['exp_name']
    filename = os.path.join('outputs/classification/', filename)
    push_latest_model(filename, config['exp_config']['exp_name'])
    
for e in hierarchical_experiments :
    config = e(data, structured=False)
    filename = config['exp_config']['exp_name']
    filename = os.path.join('outputs/hierarchical_classification/', filename)
    push_latest_model(filename, config['exp_config']['exp_name'])
    
    config = e(data, structured=True)
    filename = config['exp_config']['exp_name']
    filename = os.path.join('outputs/hierarchical_classification/', filename)
    push_latest_model(filename, config['exp_config']['exp_name'])
    
for e in structured_experiments :
    config = e(data, structured=True, encodings=['gender_y', 'ethnicity_y', 'age_y'])
    filename = config['exp_config']['exp_name']
    filename = os.path.join('outputs/classification/', filename)
    push_latest_model(filename, config['exp_config']['exp_name'])
    
    config = e(data, structured=False, encodings=['gender_y', 'ethnicity_y', 'age_y'])
    filename = config['exp_config']['exp_name']
    filename = os.path.join('outputs/classification/', filename)
    push_latest_model(filename, config['exp_config']['exp_name'])
    
    config = e(data, structured=True, encodings=data.structured_columns)
    filename = config['exp_config']['exp_name']
    filename = os.path.join('outputs/classification/', filename)
    push_latest_model(filename, config['exp_config']['exp_name'])
    
    config = e(data, structured=False, encodings=data.structured_columns)
    filename = config['exp_config']['exp_name']
    filename = os.path.join('outputs/classification/', filename)
    push_latest_model(filename, config['exp_config']['exp_name'])
    
    
for e in os.listdir('outputs/baselines/Readmission_y/baselines/') :
    filename = os.path.join('outputs/baselines/Readmission_y/baselines/', e)
    push_latest_model(filename, os.path.join('Readmission_y/baselines/', e))

In [ ]:
dataset = 'Readmission_y'
dataset_path = os.path.join('latex_evals', dataset)
output_path = os.path.join('Text-encoding-EHR/results/', dataset)
os.makedirs(output_path, exist_ok=True)

In [ ]:
dirs = os.listdir(dataset_path)
keys_to_use = ['1/precision', '1/recall', '1/f1-score', 'accuracy', 'roc_auc', 'pr_auc']
for d in dirs :
    subpath = os.path.join(dataset_path, d)
    output_file = os.path.join(output_path, d + '.csv')
    dfs = []
    for f in sorted(os.listdir(subpath)) :
        if os.path.isfile(os.path.join(subpath, f)) :
            d = json.load(open(os.path.join(subpath, f)))
            results = {k:d['results'][k] for k in keys_to_use}
            results['Method'] = f[:-14].replace('+', ' +').replace('_', ':')
            dfs.append(pd.DataFrame([results]))
        else :
            logging.error("%s not a file", f)
    
    dfs = pd.concat(dfs)
    dfs.to_csv(output_file, columns=['Method'] + keys_to_use, index=False)